In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

In [7]:
#create a list of all the yc batches
batches = ['s05', 'w06', 's06', 'w07', 's07', 'w08', 's08', 'w09', 's09', 'w10', 's10', 'w11', 's11', 'w12', 's12', 'w13', 's13', 'w14', 's14', 'w15', 's15', 'w16', 's16', 'w17', 's17', 'w18', 's18', 'w19', 's19', 'w20']

In [55]:
def get_companies(batch):
    url = f'https://www.ycdb.co/batch/{batch}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    companies = []
    # Extract the rows from the table
    rows = table.find_all('tr')

    for row in rows:
        # Extract company name and growth score from each row
        columns = row.find_all('td')
        if columns:
            companies.append(name_clean(columns[0].a.text.strip()))

    return companies

def name_clean(name):
    #replace all spaces with dash and lowercase the name
    name = name.replace(' ', '-').lower()
    return name

In [56]:
get_companies('s05')

['clickfacts',
 'infogami',
 'kiko',
 'loopt',
 'memamp',
 'parakey',
 'posthaven',
 'reddit',
 'simmery',
 'textpayme',
 'y-combinator']

In [57]:
# for each batch, get all the companies, and add to the master list
all_companies = []
for batch in batches:
    companies = get_companies(batch)
    all_companies.extend(companies)


In [58]:
all_companies

['clickfacts',
 'infogami',
 'kiko',
 'loopt',
 'memamp',
 'parakey',
 'posthaven',
 'reddit',
 'simmery',
 'textpayme',
 'y-combinator',
 'audiobeta',
 'clustrix',
 'flagr',
 'inkling',
 'project-wedding',
 'snipshot',
 'wufoo',
 'jamglue',
 'jumpchat',
 'likebetter',
 'omgpop',
 'pollground',
 'scribd',
 'shoutfit',
 'talkito',
 'thinkature',
 'xobni',
 'zanbazaar',
 'auctomatic',
 'buxfer',
 'heysan',
 'octopart',
 'socialmoth',
 'tsumobi',
 'twitch',
 'view3',
 'virtualmin',
 'weebly',
 'whitenoise',
 'writewith',
 'zenter',
 'anywhere.fm',
 'appjet',
 'biographicon',
 'blue-frog-gaming',
 'bountii',
 'clickpass',
 'contestmachine',
 'disqus',
 'dropbox',
 'eivod',
 'fuzzwich',
 'hello-chair',
 'ijigg',
 'meetyu',
 'reble',
 'socialpicks',
 'songkick',
 'splashup',
 'zumodrive',
 '280-north',
 'a-thinking-ape',
 'bitsample',
 'deluux',
 'fathomdb',
 'heroku',
 'insoshi',
 'joberator',
 'kirkland-north',
 'machine-zone',
 'mightyquiz',
 'mixwit',
 'ninite',
 'omnisio',
 'rescuetime'